In [ ]:
import distributed

client = distributed.Client()
client

In [ ]:
import fsspec
import xarray as xr
import xdggs
from regridding import aggregation_regridding

In [ ]:
url = "https://osi-saf.ifremer.fr/sst/l2p/global/avhrr_metop_b/2022/131/20220511231303-OSISAF-L2P_GHRSST-SSTsubskin-AVHRR_SST_METOP_B-sstmgr_metop01_20220511_231303-v02.0-fv01.0.nc"

In [ ]:
fs = fsspec.filesystem("http")
ds = xr.open_dataset(fs.open(url), engine="h5netcdf", chunks={}, decode_timedelta=True)
ds

In [ ]:
grid_info = xdggs.HealpixInfo(level=9, indexing_scheme="nested")

In [ ]:
regridded = aggregation_regridding(grid_info, ds)
regridded

In [ ]:
computed = regridded.compute()
computed

In [ ]:
computed["sea_surface_temperature"].dggs.explore()